# Patch daily with jura

In [ ]:
import os
import numpy as np
from astropy.table import Table, join
from astropy.io import fits
# from desispec.io import read_table
from desispec.io.meta import faflavor2program
from specprodDB.util import cameraid

In [ ]:
daily_tiles_file = os.path.join(os.environ['DESI_SPECTRO_REDUX'], 'daily', 'tiles-daily.csv')
daily_exposures_file = os.path.join(os.environ['DESI_SPECTRO_REDUX'], 'daily', 'exposures-daily.fits')
jura_tiles_file = os.path.join(os.environ['DESI_SPECTRO_REDUX'], 'jura', 'tiles-jura.csv')
jura_exposures_file = os.path.join(os.environ['DESI_SPECTRO_REDUX'], 'jura', 'exposures-jura.fits')
daily_tiles = Table.read(daily_tiles_file, format='ascii.csv')
daily_exposures = Table.read(daily_exposures_file, format='fits', hdu='EXPOSURES')
# daily_exposures = read_table(daily_exposures_file, ext='EXPOSURES')
daily_frames = Table.read(daily_exposures_file, format='fits', hdu='FRAMES')
# daily_frames = read_table(daily_exposures_file, ext='FRAMES')
jura_tiles = Table.read(jura_tiles_file, format='ascii.csv')
jura_exposures = Table.read(jura_exposures_file, format='fits', hdu='EXPOSURES')
# jura_exposures = read_table(jura_exposures_file, ext='EXPOSURES')
jura_frames = Table.read(jura_exposures_file, format='fits', hdu='FRAMES')
# jura_frames = read_table(jura_exposures_file, ext='FRAMES')

## Do some QA on jura

In [ ]:
for c in jura_frames.colnames:
    if hasattr(jura_frames[c], 'mask'):
        print(c)

In [ ]:
jura_exposures['MJD'].min()

In [ ]:
jura_exposures_bad_rows = list()
bad_columns = list()
for c in jura_exposures.colnames:
    if hasattr(jura_exposures[c], 'mask'):
        print(c)
        bad_columns.append(c)
        jura_exposures_bad_rows.append(np.where(jura_exposures[c].mask)[0])
    elif jura_exposures[c].dtype.kind == 'f' and not np.isfinite(jura_exposures[c]).all():
        print(c)
        bad_columns.append(c)
        jura_exposures_bad_rows.append(np.where(~np.isfinite(jura_exposures[c]))[0])
    else:
        pass
jura_exposures_bad_rows = np.unique(np.hstack(jura_exposures_bad_rows))
jura_exposures[jura_exposures_bad_rows]

### All exposures for the tiles that contain bad exposures

In [ ]:
jura_exposures_bad_tiles = np.in1d(jura_exposures['TILEID'], jura_exposures[jura_exposures_bad_rows]['TILEID'])
jura_exposures[jura_exposures_bad_tiles]

In [ ]:
jura_tiles_bad_rows = np.in1d(jura_tiles['TILEID'], jura_exposures[jura_exposures_bad_rows]['TILEID'])
jura_tiles[jura_tiles_bad_rows]

## Find jura exposures not in daily, daily exposures not in jura

In [ ]:
assert (np.unique(jura_exposures['EXPID']) == sorted(jura_exposures['EXPID'])).all()
assert (np.unique(daily_exposures['EXPID']) == sorted(daily_exposures['EXPID'])).all()
assert (np.unique(jura_frames['EXPID']) == sorted(jura_exposures['EXPID'])).all()
assert (np.unique(daily_frames['EXPID']) == sorted(daily_exposures['EXPID'])).all()

In [ ]:
first_jura_exposure, last_jura_exposure = jura_exposures['EXPID'].min(), jura_exposures['EXPID'].max()
first_jura_exposure, last_jura_exposure

In [ ]:
first_jura_night = jura_exposures['NIGHT'][jura_exposures['EXPID'] == first_jura_exposure].min()
last_jura_night = jura_exposures['NIGHT'][jura_exposures['EXPID'] == last_jura_exposure].max()
first_jura_night, last_jura_night

In [ ]:
jura_tiles['LASTNIGHT'].min(), jura_tiles['LASTNIGHT'].max()

In [ ]:
jura_expid_set = frozenset(jura_exposures['EXPID'].tolist())
daily_expid_set = frozenset(daily_exposures['EXPID'].tolist())

In [ ]:
jura_not_in_daily = jura_expid_set - daily_expid_set
jura_not_in_daily

In [ ]:
daily_not_in_jura = daily_expid_set - jura_expid_set
# daily_not_in_jura

## Patch frames

We don't necessarily want to change the values in the `FRAMES` table, just make sure it is consistent with the `EXPOSURES` table.

In [ ]:
daily_frames_join = Table()
daily_frames_join['FRAMEID'] = np.array([100*row['EXPID'] + cameraid(row['CAMERA']) for row in daily_frames])
daily_frames_join['DAILY_INDEX'] = np.arange(len(daily_frames))

In [ ]:
jura_frames_join = Table()
jura_frames_join['FRAMEID'] = np.array([100*row['EXPID'] + cameraid(row['CAMERA']) for row in jura_frames])
jura_frames_join['JURA_INDEX'] = np.arange(len(jura_frames))

In [ ]:
j = join(daily_frames_join, jura_frames_join, join_type='outer', keys='FRAMEID')
# j

In [ ]:
daily_frames_index = j[(~j['JURA_INDEX'].mask) & (~j['DAILY_INDEX'].mask)]['DAILY_INDEX']
jura_frames_index = j[(~j['JURA_INDEX'].mask) & (~j['DAILY_INDEX'].mask)]['JURA_INDEX']

In [ ]:
daily_frames_patched = daily_frames.copy()
for column in daily_frames_patched.colnames:
    if hasattr(daily_frames_patched[column], 'mask') and not column.startswith('TSNR2_'):
        if np.any(daily_frames_patched[column].mask[daily_frames_index]):
            print("Patching {0:d} rows in dst_frames column {1}.".format(np.sum(daily_frames_patched[column].mask[daily_frames_index]), column))
            jura_frames_matched = jura_frames[column][jura_frames_index]
            daily_frames_matched = daily_frames_patched[column][daily_frames_index]
            daily_frames_mask_matched = daily_frames_patched[column].mask[daily_frames_index]
            assert np.sum(daily_frames_mask_matched) == np.sum(daily_frames_patched[column].mask[daily_frames_index])
            daily_frames_matched[daily_frames_mask_matched] = jura_frames_matched[daily_frames_mask_matched]
            daily_frames_matched.mask[daily_frames_mask_matched] = False
            daily_frames_patched[column][daily_frames_index] = daily_frames_matched
            daily_frames_patched[column].mask[daily_frames_index] = daily_frames_matched.mask
            # print(type(daily_frames_patched[column].data.data))
            assert not (daily_frames_patched[column].data.data == daily_frames[column].data.data).all()
daily_frames_patched[daily_frames['SEEING_ETC'].mask]

## Patch exposures

We want to only *patch* and rows that:

* Appear in `jura`.
* Have `NIGHT >= first_jura_night`.
* Have `EFFTIME_SPEC > 0`.

We want to only *load* rows that:

* Have `NIGHT >= first_jura_night`.
* Have `EFFTIME_SPEC > 0`.

which is slightly different. However we don't want to *remove* rows that *don't* satisfy these criteria.

In [ ]:
daily_exposures_join = Table()
daily_exposures_join['EXPID'] = daily_exposures['EXPID']
# daily_exposures_join['NIGHT'] = daily_exposures['NIGHT']
# daily_exposures_join['EFFTIME_SPEC'] = daily_exposures['EFFTIME_SPEC']
daily_exposures_join['DAILY_INDEX'] = np.arange(len(daily_exposures))

In [ ]:
jura_exposures_join = Table()
jura_exposures_join['EXPID'] = jura_exposures['EXPID']
jura_exposures_join['JURA_INDEX'] = np.arange(len(jura_exposures))

In [ ]:
j = join(daily_exposures_join, jura_exposures_join, join_type='outer', keys='EXPID')
j

In [ ]:
daily_exposures_index = j[(~j['JURA_INDEX'].mask) & (~j['DAILY_INDEX'].mask)]['DAILY_INDEX']
jura_exposures_index = j[(~j['JURA_INDEX'].mask) & (~j['DAILY_INDEX'].mask)]['JURA_INDEX']

In [ ]:
daily_exposures_patched = daily_exposures.copy()
can_patch = ('NIGHT', 'EXPID', 'TILEID', 'TILERA', 'TILEDEC', 'MJD', 'SURVEY', 'PROGRAM', 'FAPRGRM', 'FAFLAVOR', 'EXPTIME', 'GOALTIME', 'GOALTYPE', 'MINTFRAC', 'AIRMASS', 'EBV', 'SEEING_ETC', 'EFFTIME_ETC',
             'TRANSPARENCY_GFA', 'SEEING_GFA', 'FIBER_FRACFLUX_GFA', 'FIBER_FRACFLUX_ELG_GFA', 'FIBER_FRACFLUX_BGS_GFA', 'FIBERFAC_GFA', 'FIBERFAC_ELG_GFA', 'FIBERFAC_BGS_GFA', 'AIRMASS_GFA', 'SKY_MAG_AB_GFA',
             'EFFTIME_GFA', 'EFFTIME_DARK_GFA', 'EFFTIME_BRIGHT_GFA', 'EFFTIME_BACKUP_GFA')
for column in ['TILERA', 'TILEDEC', 'MJD', 'SURVEY'] + [c for c in daily_exposures_patched.colnames if hasattr(daily_exposures_patched[c], 'mask') and c in can_patch]:
    print(f"Patching {column}...")
    if hasattr(jura_exposures[column], 'mask'):
        if np.any(jura_exposures[column].mask[jura_exposures_index]):
            jura_exposures[column][jura_exposures[column].mask] = 0
            jura_exposures[column].mask[jura_exposures[column].mask] = False
    daily_exposures_patched[column][daily_exposures_index] = jura_exposures[column][jura_exposures_index]
    if hasattr(daily_exposures_patched[column], 'mask'):
        daily_exposures_patched[column].mask[daily_exposures_index] = False

known_bad_exposures = np.in1d(daily_exposures_patched['EXPID'], np.array([74307, 79769, 83420, 110852, 190752]))
daily_exposures_patched[known_bad_exposures]

### After patching are there still missing data?

In [ ]:
assert not (daily_exposures_patched['TILERA'] == daily_exposures['TILERA']).all()
assert not (daily_exposures_patched['TILEDEC'] == daily_exposures['TILEDEC']).all()
assert not (daily_exposures_patched['MJD'] == daily_exposures['MJD']).all()
assert not (daily_exposures_patched['SURVEY'] == daily_exposures['SURVEY']).all()
assert (daily_exposures_patched['PROGRAM'] == daily_exposures['PROGRAM']).all()
assert (daily_exposures_patched['FAPRGRM'] == daily_exposures['FAPRGRM']).all()
assert (daily_exposures_patched['FAFLAVOR'] == daily_exposures['FAFLAVOR']).all()

In [ ]:
missing_mjd = np.where((daily_exposures_patched['NIGHT'] >= first_jura_night) & (daily_exposures_patched['EFFTIME_SPEC'] > 0) & (daily_exposures_patched['MJD'] < 50000))[0]
for row in daily_exposures_patched[missing_mjd]:
    raw_data_file = os.path.join(os.environ['DESI_SPECTRO_DATA'], "{0:08d}".format(row['NIGHT']), "{0:08d}".format(row['EXPID']), "desi-{0:08d}.fits.fz".format(row['EXPID']))
    with fits.open(raw_data_file, mode='readonly') as hdulist:
        mjd_obs = hdulist['SPEC'].header['MJD-OBS']
    print("INFO: tile {0:d} exposure {1:d} has MJD-OBS = {2:f} in {3}!".format(row['TILEID'], row['EXPID'], mjd_obs, raw_data_file))
    w = np.where(daily_exposures_patched['EXPID'] == row['EXPID'])[0]
    assert len(w) == 1
    daily_exposures_patched['MJD'][w] = mjd_obs
    print(daily_exposures_patched[w][['NIGHT', 'EXPID', 'TILEID', 'MJD']])

In [ ]:
still_missing_mjd = np.where((daily_exposures_patched['NIGHT'] >= first_jura_night) & (daily_exposures_patched['EFFTIME_SPEC'] > 0) & (daily_exposures_patched['MJD'] < 50000))[0]
daily_exposures_patched[still_missing_mjd]

In [ ]:
possible_expid_still_patchable = list()
for c in daily_exposures_patched.colnames:
    if hasattr(daily_exposures_patched[c], 'mask'):
        if daily_exposures_patched[c].mask.any():
            n_masked = daily_exposures_patched[c].mask.sum()
            not_in_jura = [e in daily_not_in_jura for e in daily_exposures_patched['EXPID'][daily_exposures_patched[c].mask]]
            if all(not_in_jura):
                print(f"Column {c} still has {n_masked:d} masked values, but the exposures are not in jura and cannot be patched.")
            else:
                if c in can_patch:
                    for e in daily_exposures_patched['EXPID'][daily_exposures_patched[c].mask]:
                        if e not in daily_not_in_jura:
                            possible_expid_still_patchable.append(int(e))
                    print(f"Column {c} still has {n_masked:d} masked values, and some exposures can still be patched.")
                else:
                    print(f"Column {c} still has {n_masked:d} masked values, and some exposures could still be patched, but they do not meet the patchable column guidelines.")

possible_expid_still_patchable = np.unique(np.array(possible_expid_still_patchable))
daily_exposures_patched[np.in1d(daily_exposures_patched['EXPID'], possible_expid_still_patchable)]

### Fill remaining masked values with zero.

In [ ]:
for c in daily_exposures_patched.colnames:
    if hasattr(daily_exposures_patched[c], 'mask'):
        if daily_exposures_patched[c].mask.any():
            daily_exposures_patched[c][daily_exposures_patched[c].mask] = 0
            daily_exposures_patched[c].mask[daily_exposures_patched[c].mask] = False

## Patch tiles

Similar to the discussion above, we want `LASTNIGHT >= first_jura_night`.

In [ ]:
assert (np.unique(jura_exposures['TILEID']) == sorted(jura_tiles['TILEID'])).all()
assert (np.unique(daily_exposures['TILEID']) == sorted(daily_tiles['TILEID'])).all()

### First patch PROGRAM with faflavor2program()

In [ ]:
daily_tiles_patched = daily_tiles.copy()
daily_tiles_patched['PROGRAM'] = faflavor2program(daily_tiles_patched['FAFLAVOR'])

In [ ]:
oddball_survey = np.where((daily_tiles_patched['SURVEY'] != 'cmx') & (daily_tiles_patched['SURVEY'] != 'sv1') & (daily_tiles_patched['SURVEY'] != 'sv2') & (daily_tiles_patched['SURVEY'] != 'sv3') & (daily_tiles_patched['SURVEY'] != 'main') & (daily_tiles_patched['SURVEY'] != 'special'))[0]
oddball_program = np.where((daily_tiles_patched['PROGRAM'] != 'backup') & (daily_tiles_patched['PROGRAM'] != 'bright') & (daily_tiles_patched['PROGRAM'] != 'dark') & (daily_tiles_patched['PROGRAM'] != 'other'))[0]

In [ ]:
assert (daily_tiles['SURVEY'][oddball_survey] == 'unknown').all()
assert len(oddball_program) == 0

In [ ]:
unknown_cmx = np.where((daily_tiles['SURVEY'] == 'unknown') & (daily_tiles['FAFLAVOR'] != 'unknown'))[0]
print(daily_tiles[['TILEID', 'SURVEY', 'PROGRAM', 'FAPRGRM', 'FAFLAVOR', 'GOALTYPE', 'LASTNIGHT']][unknown_cmx])

In [ ]:
unknown_unknown = np.where((daily_tiles['SURVEY'] == 'unknown') & (daily_tiles['FAFLAVOR'] == 'unknown'))[0]
daily_tiles['LASTNIGHT'][unknown_unknown].max()

In [ ]:
print(daily_tiles[['TILEID', 'SURVEY', 'PROGRAM', 'FAPRGRM', 'FAFLAVOR', 'GOALTYPE', 'LASTNIGHT']][unknown_unknown])

In [ ]:
# For now just patch with 'cmx'
daily_tiles['SURVEY'][oddball_survey] = 'cmx'

## Write out the patched files

In [ ]:
daily_tiles_patched.write(os.path.join(os.environ['DESI_ROOT'], 'users', os.environ['USER'], 'tiles-daily-patched-with-jura.csv'), format='ascii.csv', overwrite=True)

In [ ]:
daily_exposures_patched.write(os.path.join(os.environ['DESI_ROOT'], 'users', os.environ['USER'], 'exposures-daily-patched-with-jura.csv'), format='ascii.csv', overwrite=True)

In [ ]:
daily_exposures_fits = fits.HDUList([fits.PrimaryHDU(), fits.table_to_hdu(daily_exposures_patched), fits.table_to_hdu(daily_frames_patched)])

In [ ]:
daily_exposures_fits.info()

In [ ]:
daily_exposures_fits.writeto(os.path.join(os.environ['DESI_ROOT'], 'users', os.environ['USER'], 'exposures-daily-patched-with-jura.fits'), overwrite=True)